ImageDataGenerator 사용, 다중 gpu 로드율이 낮아 문제

In [1]:
import numpy as np
import pandas as pd
import os, random, csv
import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models 
from tensorflow.keras.layers import BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras import optimizers

# MULTI_GPU
from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

/compuworks/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/compuworks/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/compuworks/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/compuworks/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12461016566923179033
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12946637124623115424
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10823575143
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17702293837049850538
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:40:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 10823575143
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9804005277108675290
physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:41:00.0, compute capability: 7.5"
, name: "/device:GPU:2"
device_type: "GPU"
memory_limit: 10823575143
locality {
  bus_id: 1
  links {
  }
}
incarnation: 492276458642565010
physical_device_desc: "device: 2, name: NVIDIA GeForce 

In [2]:
# Define image path
train_dir = './stanford/train'
test_dir = './stanford/test'

test_datagen = ImageDataGenerator(rescale=1./255)

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지를 150 × 150 크기로
        target_size=(224,224),
        batch_size=32*6,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224,224),
        batch_size=32*6,
        class_mode='categorical')



Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [3]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)
    print('배치 레이블 크기:', labels_batch.shape)
    break

배치 데이터 크기: (192, 224, 224, 3)
배치 레이블 크기: (192, 120)


In [4]:

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150,150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(120, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])


In [5]:
model = models.Sequential()
model.add(ResNet50(include_top=True, weights=None, input_shape=(224, 224, 3)))

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(120, activation='softmax'))

model = multi_gpu_model(model, gpus=6)

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
resnet50_input (InputLayer)     (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 224, 224, 3)  0           resnet50_input[0][0]             
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 224, 224, 3)  0           resnet50_input[0][0]             
__________________________________________________________________________________________________
lambda_2 (Lambda)    

In [ ]:
history = model.fit_generator(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    use_multiprocessing=True,
    workers=6,
    )

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
63/63 [==============================] - 202s 3s/step - loss: 4.7825 - acc: 0.0108 - val_loss: 4.7955 - val_acc: 0.0092
Epoch 2/30
63/63 [==============================] - 93s 1s/step - loss: 4.7258 - acc: 0.0148 - val_loss: 4.6728 - val_acc: 0.0117
Epoch 3/30
63/63 [==============================] - 94s 1s/step - loss: 4.6757 - acc: 0.0146 - val_loss: 4.6806 - val_acc: 0.0110
Epoch 4/30
21/63 [=========>....................] - ETA: 41s - loss: 4.6288 - acc: 0.0179

In [ ]:
model.save('./stanford/dog_breed_model_1.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()